# Neural Net testing

In [ ]:
import numpy as np                                                              
import matplotlib.pyplot as plt
%matplotlib inline

import neuralNet as nn
import utils as u

## MNIST

In [2]:
from torchvision import datasets
train_dataset = datasets.MNIST('./data', train=True, download=True)
test_dataset = datasets.MNIST('./data', train=False, download=True)
X = train_dataset.data.numpy()
Y = train_dataset.targets.numpy()
tX = test_dataset.data.numpy()
tY = test_dataset.targets.numpy()

X, Y = X.reshape([X.shape[0], -1]), u.onehot(Y)
tX, tY = tX.reshape([tX.shape[0], -1]), u.onehot(tY)

In [ ]:
model = nn.NeuralNet(sizes=[X.shape[1], 20, 10], loss='categorical', nonLin=['relu','softmax'])
it, E_ins = model.learn(X, Y, maxIters=2, trackE_in=True)

/Users/akshayyeluri/git/MLModels/neuralNet.py:27: RuntimeWarning: invalid value encountered in true_divide
  'f' : lambda s :  np.exp(s) / np.exp(s).sum(), # softmax activation function
/Users/akshayyeluri/git/MLModels/neuralNet.py:28: RuntimeWarning: invalid value encountered in true_divide
  'df': lambda s :  np.diag(np.exp(s)/np.exp(s).sum()) - \
/Users/akshayyeluri/anaconda3/envs/neural/lib/python3.6/site-packages/numpy/core/numeric.py:1129: RuntimeWarning: invalid value encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
/Users/akshayyeluri/git/MLModels/neuralNet.py:29: RuntimeWarning: invalid value encountered in true_divide
  np.outer(np.exp(s), np.exp(s)) / (np.exp(s).sum() ** 2)
/Users/akshayyeluri/git/MLModels/neuralNet.py:11: RuntimeWarning: divide by zero encountered in true_divide
  'df': lambda t,y: -y / t
/Users/akshayyeluri/git/MLModels/neuralNet.py:11: RuntimeWarning: invalid value encountered in true_divide
  'df': lambda t,y: 

# Softmax Test

In [ ]:
f, line1 = u.genF(zero_one=True) # We want labels to be 0/1 not -1/1
X, Y = u.genData(f, 50)

In [ ]:
def nn_plot_helper(model, X, Y, E_ins, line1, transform = lambda w: [*w[0][1:], w[0][0]]):
    fig, ax = plt.subplots(2, 1, figsize=(8,8))
    u.plotE_ins(E_ins, axis=ax[0])
    u.plotLine(*line1, axis=ax[1], label='Target')
    u.plotLine(*transform(model.weights), color='g', \
                axis=ax[1], label='Hypothesis')
    inds = (Y == 1)                                
    ax[1].plot(X[inds, -2], X[inds, -1], 'b+')                   
    ax[1].plot(X[~inds, -2], X[~inds, -1], 'r_')
    ax[1].set_xlabel('X1'); ax[1].set_ylabel('X2')
    ax[1].set_xlim([-1, 1]); ax[1].set_ylim([-1, 1])
    ax[1].legend()
    return ax

In [ ]:
model = nn.NeuralNet(sizes=[3, 1], loss='log', nonLin='sigmoid')
it, E_ins = model.learn(X, Y, maxIters=500, trackE_in=True)
nn_plot_helper(model, X, Y, E_ins, line1);

In [ ]:
one_hot = lambda Y: np.hstack((Y[:, None], (1 - Y)[:, None]))
def transform(w):
    w_p = w[0][:, 0] - w[0][:, 1]
    return [*w_p[1:], w_p[0]]
    
model2 = nn.NeuralNet(sizes=[3, 2], loss='categorical', nonLin='softmax')
it, E_ins = model2.learn(X, one_hot(Y), maxIters=500, trackE_in=True)
nn_plot_helper(model2, X, Y, E_ins, line1, transform = transform);

## One variable function testing

In [ ]:
testFunc = lambda x: np.sin(2 * x)

inputs = np.linspace(-np.pi, np.pi, 200)
outputs = testFunc(inputs)

sizes = [1, 100, 100, 1]

## For cosine, need different nonlinearity
#model = nn.NeuralNet(sizes, nonLin='sigmoid')
model = nn.NeuralNet(sizes, nonLin='tanh')

it, E_ins = model.learn(inputs, outputs, maxIters=1000, trackE_in=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8))

u.plotE_ins(E_ins, axis=ax[0])
model.quickPlot(inputs, outputs, axis=ax[1])

fig.show()

In [ ]:
inputs = np.random.uniform(-np.pi, np.pi, 20)
outputs = testFunc(inputs)

outputs2 = np.array([model.calculate(i) for i in inputs])
outputs2 - outputs

## Multi-Variable Function Testing

In [ ]:
testFunc = lambda x, y: np.cos(x + y)

inputs = np.linspace(-np.pi / 2, np.pi / 2, 100)
inputs = np.vstack((inputs / 3, inputs * 2 / 3)).T
outputs = testFunc(inputs[:, 0], inputs[:, 1])

sizes = [2, 100, 100, 1]

model = nn.NeuralNet(sizes, nonLin='sigmoid')
#model = nn.NeuralNet(sizes)

it, E_ins = model.learn(inputs, outputs, maxIters=1000, trackE_in=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 8))

u.plotE_ins(E_ins, axis=ax[0])

ax[1].plot(np.sum(inputs, axis=1), [model.calculate(i) for i in inputs], color='g', label='Net Outputs')
ax[1].plot(np.sum(inputs, axis=1), outputs, color='k', label='Real Outputs')
ax[1].set_xlabel('input')
ax[1].set_ylabel('output')
ax[1].legend()

fig.show()

In [ ]:
inputs = np.random.uniform(-np.pi, np.pi, 20)
inputs = np.vstack((inputs / 3, inputs * 2 / 3)).T
outputs = testFunc(inputs[:, 0], inputs[:, 1])

outputs2 = np.array([model.calculate(i) for i in inputs])
outputs2 - outputs

# XOR (doesn't work)

In [ ]:
testFunc = lambda x: np.sum(np.atleast_2d(x), axis=1) % 2

n_max = 1024
nDigs = len(bin(n_max)[2:])
inputs = np.array([[int(i) for i in list(bin(x)[2:].zfill(nDigs))] for x in range(n_max)])
outputs = testFunc(inputs)

sizes = [nDigs, 100, 1]

model = nn.NeuralNet(sizes, loss='log', nonLin='sigmoid')

it, E_ins = model.learn(inputs, outputs, maxIters=100, trackE_in=True)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

u.plotE_ins(E_ins, axis=ax)
#model.quickPlot(inputs, outputs, axis=ax[1])

fig.show()

outputs2 = np.round(np.array([model.calculate(i) for i in inputs]))